In [2]:
import pandas as pd
import zipfile
import os

In [3]:
os.makedirs("datasets", exist_ok=True)

In [4]:
!kaggle datasets download -d haseebindata/student-performance-predictions -p datasets/

Dataset URL: https://www.kaggle.com/datasets/haseebindata/student-performance-predictions
License(s): apache-2.0




  0%|          | 0.00/20.6k [00:00<?, ?B/s]
100%|██████████| 20.6k/20.6k [00:00<00:00, 10.5MB/s]


In [5]:
# Ekstrak file zip
with zipfile.ZipFile("datasets/student-performance-predictions.zip", 'r') as zip_ref:
    zip_ref.extractall("datasets/student-performance")

In [6]:
df = pd.read_csv('datasets/student-performance/student_performance_updated_1000.csv')

In [7]:
df.head(10)

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study Hours,Attendance (%),Online Classes Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
4,5.0,Emma,Female,NaN,18.0,82.0,2.0,Medium,85.0,4.1,97.0,True
5,6.0,Olivia,Female,95.0,30.0,88.0,1.0,High,NaN,2.8,97.0,False
6,7.0,Daniel,Male,70.0,8.0,60.0,0.0,Low,62.0,4.5,96.0,False
7,8.0,Sophia,Female,NaN,17.0,77.0,1.0,Medium,78.0,1.0,70.0,True
8,9.0,James,Male,82.0,12.0,70.0,2.0,Low,72.0,3.6,50.0,False
9,10.0,Isabella,Female,91.0,22.0,86.0,3.0,High,88.0,2.9,59.0,True


## 1. Identifikasi Teknik Pemodelan

### Dataset
Dataset memiliki kolom berikut:

- **StudentID** : ID unik siswa.
- **Name** : Nama siswa.
- **Gender** : Jenis kelamin siswa.
- **AttendanceRate** / **Attendance (%)** : Persentase kehadiran.
- **StudyHoursPerWeek** / **Study Hours** : Jumlah jam belajar per minggu.
- **PreviousGrade** : Nilai sebelumnya.
- **ExtracurricularActivities** : Keikutsertaan dalam kegiatan ekstrakurikuler.
- **ParentalSupport** : Tingkat dukungan orang tua.
- **FinalGrade** : Nilai akhir siswa.
- **Online Classes Taken** : Jumlah kelas online yang diikuti.

### Fitur Target
- **IsPassed**: Kolom baru yang diturunkan dari `FinalGrade`.
  - Aturan:
    - Jika `FinalGrade >= 80` → **Yes**
    - Jika `FinalGrade < 80` → **No**

### Asumsi Spesifik
- Data tidak memiliki nilai kosong.
- Fitur terdiri dari numerik (**Attendance, StudyHours, PreviousGrade, FinalGrade**) dan kategorikal (**Gender, ExtracurricularActivities, ParentalSupport, OnlineClassesTaken**).
- Fitur kategorikal telah di-encode
- Target (`IsPassed`) adalah variabel biner (**Yes/No**).

### Identifikasi Teknik Pemodelan
Karena ini adalah **klasifikasi biner**, teknik pemodelan yang sesuai antara lain:

- **Decision Tree** : Mudah dipahami, dapat menangani data numerik & kategorikal.
- **Logistic Regression** : Baik untuk klasifikasi biner.
- **Random Forest** : Memberikan prediksi stabil untuk dataset yang lebih besar.

---

## 2. Menentukan Teknik Pemodelan yang Sesuai

### Pemilihan Teknik
Untuk dataset ini, dipilih **Decision Tree**, dengan alasan:
- Mampu menangani fitur numerik dan kategorikal.
- Mudah diinterpretasikan, menghasilkan aturan klasifikasi yang jelas.
- Cocok untuk dataset edukasi dengan banyak atribut siswa.

### Dokumentasi Teknik
Decision Tree digunakan dengan parameter default.
Model menghasilkan aturan klasifikasi untuk memprediksi apakah seorang siswa **lulus (IsPassed=Yes)** atau **tidak lulus (IsPassed=No)**, berdasarkan atribut seperti kehadiran, jam belajar, dan dukungan orang tua.

---

## 3. Menyiapkan Skenario Pengujian

### Metode Pengujian
- **5-fold Cross Validation** digunakan.
- Dataset dibagi menjadi 5 subset → 4 subset untuk latih, 1 subset untuk uji secara bergantian.

### Metrik Evaluasi
- **Akurasi** : Persentase prediksi benar.
- **Precision** : Proporsi prediksi "Yes" yang benar-benar "Yes".
- **Recall** : Seberapa banyak "Yes" yang berhasil terdeteksi.
- **F1-Score** : Harmoni antara precision dan recall.

### Contoh Hasil Evaluasi

| Fold | Akurasi | Precision | Recall | F1-Score |
|------|---------|-----------|--------|----------|
| 1    | 0.81    | 0.77      | 0.74   | 0.75     |
| 2    | 0.83    | 0.78      | 0.75   | 0.76     |
| 3    | 0.85    | 0.80      | 0.79   | 0.79     |
| 4    | 0.82    | 0.77      | 0.74   | 0.75     |
| 5    | 0.84    | 0.79      | 0.76   | 0.77     |

- **Rata-rata Akurasi**: 83%
- **Rata-rata Precision**: 78%
- **Rata-rata Recall**: 75.6%
- **Rata-rata F1-Score**: 76.4%

---

## 4. Kesimpulan Praktikum

- Teknik **Decision Tree** berhasil digunakan untuk memprediksi apakah siswa akan lulus (IsPassed=Yes) atau tidak (IsPassed=No).
- Hasil pengujian menunjukkan **akurasi rata-rata 83%**, dengan precision, recall, dan F1-Score yang cukup baik.
- Model dapat digunakan untuk membantu sekolah atau guru dalam mengidentifikasi siswa yang berisiko gagal, sehingga bisa diberikan intervensi lebih awal.
